In [ ]:
from transformers import set_seed
set_seed(42)

# Load Models

##  Projector (independen)

In [ ]:
import torch
projector_ckpt = torch.load("../output/finetuned/2025-07-19_projector_learningrate_1e-3/projector_checkpoints/epoch_10/projector.pth")
projector_ckpt

## Qwen3

In [ ]:
import sys
sys.path.append('..')
from transformers import AutoConfig
from model.xQwen3 import XQwen3Config, XQwen3ForCausalLM
model_name_or_path  = "Qwen/Qwen3-1.7B"
config = XQwen3Config.from_pretrained(model_name_or_path, retriever_hidden_size=384)
fresh_model = XQwen3ForCausalLM.from_pretrained(  # XLlamaForCausalLM
    model_name_or_path,
    config=config,
    torch_dtype = 'bfloat16'
).to("cuda:0")

## Xqwen3 (projector+qwen3 sepaket)

In [ ]:
import torch
import sys
sys.path.append('..')
from model.xQwen3 import XQwen3Config, XQwen3ForCausalLM

# Model hasil fine-tuning
finetuned_model = XQwen3ForCausalLM.from_pretrained(
    "../output/finetuned/2025-07-19_projector_learningrate_1e-3/finished_model",
    torch_dtype=torch.bfloat16
).to("cuda:0")

# EDA

## Membuktikan bahwa yang fresh dgn finetuned berbeda (finetuning berhasil)

In [ ]:
layers_to_check = [
    "projector.projector.0.weight",
    "projector.projector.0.bias",
    "projector.projector.2.weight",
    "projector.projector.2.bias",
]

for layer in layers_to_check:
    finetuned_tensor = dict(finetuned_model.named_parameters())[layer].detach().cpu().float()
    fresh_tensor = dict(fresh_model.named_parameters())[layer].detach().cpu().float()

    is_equal = torch.allclose(finetuned_tensor, fresh_tensor, atol=1e-6)
    max_diff = torch.max(torch.abs(finetuned_tensor - fresh_tensor)).item()

    print(f"{layer}:")
    print(f"  ➤ Equal? {is_equal}")
    print(f"  ➤ Max abs diff: {max_diff:.6f}")


### Membuktikan bahwa bisa me-load projector finetuned ke xqwen3

In [ ]:
fresh_projector_state = {
    k: v.detach().clone().cpu().float() 
    for k, v in fresh_model.projector.projector.state_dict().items()
}
fresh_projector_state.keys()

# Langkah 3: Injeksi ke dalam model
fresh_model.projector.projector.load_state_dict(projector_ckpt)

In [ ]:
print("Perbandingan bobot projector fresh vs yang sudah difinetuned:")
for name in projector_ckpt:
    before = fresh_projector_state[name]
    after = fresh_model.projector.projector.state_dict()[name].detach().cpu().float()

    if torch.allclose(before, after, atol=1e-6):
        print(f"  🔁 {name}: ❌ TIDAK berubah")
    else:
        diff = torch.max(torch.abs(before - after)).item()
        print(f"  ✅ {name}: BERUBAH (max abs diff: {diff:.6f})")